In [7]:
from bs4 import BeautifulSoup
import json
import pandas as pd
from scrapy.crawler import CrawlerProcess
from spiders.pvp_web_scraping import ProceduraSpider
import os

In [8]:
#To see the dic
''' 
with open('../tribunale_data.json', 'r') as file:
    loaded_dict = json.load(file)
loaded_dict
'''

" \nwith open('../tribunale_data.json', 'r') as file:\n    loaded_dict = json.load(file)\nloaded_dict\n"

In [13]:
class ReadHTMLParser:
    def __init__(self, tribunale, procedura, anno):
        self.file_name = f"{tribunale}_{procedura}_{anno}.html"

    def read_html(self):
        with open('output/'+self.file_name, 'r', encoding='utf-8') as file:
            html_content = file.read()
        return html_content
  
    def parse(self):
        html_content = self.read_html()
        soup = BeautifulSoup(html_content, 'html.parser')

        details_list = []
        for lotto in soup.find_all(class_='col-md-6 col-lg-4 col-sm-6 col-xs-12 tile-dettaglio'):
            address = lotto.find('div', class_='anagrafica-risultato').get_text().strip()
            nLotto = lotto.find(class_='black').get_text()

            data_vendita = lotto.find(class_='margin-top-15').find(class_='font-green').get_text()

            offerta_elements = lotto.find_all(class_='inline font-blue')
            offerta_minima = offerta_elements[0].get_text() 
            rialzo_minimo = offerta_elements[1].get_text()
            n_procedura = lotto.find(class_='font-black inline').get_text()
            prezzo_base = lotto.find(class_='margin-bottom-15').find(class_='font-blue font18').get_text()
            
            if nLotto and "Lotto nr." in nLotto:
                nLotto = nLotto.replace("\n                ", "")

            details_dict = {
                'Address': address,
                'Lotto': nLotto.strip() if nLotto else '',
                'Data di vendita': data_vendita.strip() if data_vendita else '',
                'Offerta minima': offerta_minima.strip() if offerta_minima else '',
                'Rialzo minimo': rialzo_minimo.strip() if rialzo_minimo else '',
                'N° Procedura': n_procedura.strip() if n_procedura else '',
                "Prezzo base d'asta": prezzo_base.strip() if prezzo_base else ''
            }
            details_list.append(details_dict)

        return details_list


In [10]:
inputs = pd.read_excel('input/input.xlsx')
inputs.columns = [col.lower() for col in inputs.columns]
inputs

,tribunale,procedura,anno
0,tribunale di aosta,18,2017
1,Tribunale di arezzo,32,2017


In [11]:
# pd.DataFrame(parsed_data).to_excel('scraped_data.xlsx', index=False) #saves the output normally, without column widht adjusted
def dicToExcel(dict_list):
    with pd.ExcelWriter('scraped_data.xlsx', engine='xlsxwriter') as writer:
        for dictionary in dict_list:
            sheet_name = list(dictionary.keys())[0]
            df = pd.DataFrame(dictionary[sheet_name])
            df.to_excel(writer, sheet_name=sheet_name, index=False)

            # Get the worksheet object
            worksheet = writer.sheets[sheet_name]

            # Auto-adjust column widths based on content length
            for i, col in enumerate(df.columns):
                column_len = max(df[col].astype(str).map(len).max(), len(col))
                worksheet.set_column(i, i, column_len + 2)


In [15]:
dictionaries = []

with open('tribunale_data.json', 'r') as file:
    loaded_dict = json.load(file)

for index, row in inputs.iterrows():
    tribunale = row['tribunale']
    n_tribunale = loaded_dict.get(tribunale.lower())
    procedura = row['procedura']
    anno = row['anno']    


    parser = ReadHTMLParser(n_tribunale, procedura, anno)
    parsed_data = parser.parse()
    
    dictionaries.append({f'{tribunale}_{procedura}_{anno}': parsed_data})  # Append as a dictionary
    




In [52]:
dataframe = pd.DataFrame(parsed_data)
dataframe

,Address,Lotto,Data di vendita,Offerta minima,Rialzo minimo,N° Procedura,Prezzo base d'asta
0,VIA DELL'ARTIGIANATO - 52022 Cavriglia (AR),Lotto nr.3,18/04/2018 15:00,"€ 59.000,00","€ 5.000,00",32/2017,"€ 59.000,00"
1,VIA ARTIGIANATO - 52022 Cavriglia (AR),Lotto nr.2,18/04/2018 15:00,"€ 52.000,00","€ 5.000,00",32/2017,"€ 52.000,00"
2,VIA DELL'ARTIGIANATO - 52022 Cavriglia (AR),Lotto nr.4,18/04/2018 15:00,"€ 47.000,00","€ 5.000,00",32/2017,"€ 47.000,00"
3,Località Le Giunchete - 52022 Cavriglia (AR),Lotto nr.8,18/04/2018 15:00,"€ 72.000,00","€ 5.000,00",32/2017,"€ 72.000,00"
4,"VIA LA MINIERINA, 10 - 52022 Cavriglia (AR)",Lotto nr.1,18/04/2018 15:00,"€ 350.000,00","€ 5.000,00",32/2017,"€ 350.000,00"
5,"via Rossellini, 47 - 52027 San Giovanni Valdar...",Lotto nr.5,18/04/2018 15:00,"€ 1.070.000,00","€ 10.000,00",32/2017,"€ 1.070.000,00"
6,via Montessori - 52027 San Giovanni Valdarno (AR),Lotto nr.7,18/04/2018 15:00,"€ 31.000,00","€ 1.000,00",32/2017,"€ 31.000,00"
7,VIA MONTESSORI - 52027 San Giovanni Valdarno (AR),Lotto nr.6,18/04/2018 15:00,"€ 110.000,00","€ 1.000,00",32/2017,"€ 110.000,00"
8,via rossellini - 50065 San Giovanni Valdarno (AR),LOTTO UNICO,29/08/2018 15:00,"€ 10.000,00","€ 10.000,00",32/2017,"€ 963.000,00"
9,Località le Giunchette - 52022 Cavriglia (AR),Lotto nr.5,28/02/2019 15:00,"€ 45.000,00","€ 5.000,00",32/2017,"€ 45.000,00"


In [55]:
dfs = []

for dictionary in dictionaries:
    for key_outer, value_outer in dictionary.items():
        row_data = {'id': key_outer}
        for dic in value_outer:
            for key_inner, value_inner in dic.items():
                row_data[key_inner] = value_inner
            df_temp = pd.DataFrame([row_data])
            dfs.append(df_temp)

# Concatenate the list of DataFrames into one DataFrame
df = pd.concat(dfs, ignore_index=True)

df

,id,Address,Lotto,Data di vendita,Offerta minima,Rialzo minimo,N° Procedura,Prezzo base d'asta
0,tribunale di aosta_18_2017,via chambery 182 - 11100 Aosta (AO),LOTTO UNICO,17/10/2017 11:30,"€ 67.208,70","€ 2.500,00",18/2017,"€ 89.611,60"
1,tribunale di aosta_18_2017,Via Mure - 11020 Issogne (AO),Lotto nr.1,19/06/2023 14:30,"€ 1.077.225,00","€ 0,00",18/2017,"€ 1.436.300,00"
2,tribunale di aosta_18_2017,Strada Statale n. 26 Aosta – Ivrea - 10010 Set...,Lotto nr.2,19/06/2023 14:30,"€ 15.000,00","€ 0,00",18/2017,"€ 20.000,00"
3,tribunale di aosta_18_2017,Frazione Mure snc - 11020 Issogne (AO),Lotto nr.1,27/09/2023 14:30,"€ 807.919,00","€ 0,00",18/2017,"€ 1.077.225,00"
4,tribunale di aosta_18_2017,Frazione Mure snc - 11020 Issogne (AO),Lotto nr.1,15/11/2023 14:30,"€ 605.939,00","€ 0,00",18/2017,"€ 807.919,00"
5,Tribunale di arezzo_32_2017,VIA DELL'ARTIGIANATO - 52022 Cavriglia (AR),Lotto nr.3,18/04/2018 15:00,"€ 59.000,00","€ 5.000,00",32/2017,"€ 59.000,00"
6,Tribunale di arezzo_32_2017,VIA ARTIGIANATO - 52022 Cavriglia (AR),Lotto nr.2,18/04/2018 15:00,"€ 52.000,00","€ 5.000,00",32/2017,"€ 52.000,00"
7,Tribunale di arezzo_32_2017,VIA DELL'ARTIGIANATO - 52022 Cavriglia (AR),Lotto nr.4,18/04/2018 15:00,"€ 47.000,00","€ 5.000,00",32/2017,"€ 47.000,00"
8,Tribunale di arezzo_32_2017,Località Le Giunchete - 52022 Cavriglia (AR),Lotto nr.8,18/04/2018 15:00,"€ 72.000,00","€ 5.000,00",32/2017,"€ 72.000,00"
9,Tribunale di arezzo_32_2017,"VIA LA MINIERINA, 10 - 52022 Cavriglia (AR)",Lotto nr.1,18/04/2018 15:00,"€ 350.000,00","€ 5.000,00",32/2017,"€ 350.000,00"
